<a href="https://colab.research.google.com/github/trishabhogawar/Object_Detection_Retail/blob/main/IVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install YOLOv8 and tqdm
!pip install -q ultralytics kagglehub tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.8 MB/s eta 0:00:00


In [ ]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("unidatapro/grocery-shelves")
print("Path to dataset files:", path)

In [ ]:
import os
import xml.etree.ElementTree as ET

xml_file = os.path.join(path, "annotations.xml")
labels_dir = os.path.join(path, "labels")
output_dir = os.path.join(path, "yolo_labels")
os.makedirs(output_dir, exist_ok=True)

tree = ET.parse(xml_file)
root = tree.getroot()

for image in root.findall('image'):
    img_name = image.get('name').split('/')[-1].split('.')[0]
    img_width = float(image.get('width'))
    img_height = float(image.get('height'))

    yolo_lines = []
    for polyline in image.findall('polyline'):
        points = polyline.get('points').split(';')
        x_coords = [float(p.split(',')[0]) for p in points]
        y_coords = [float(p.split(',')[1]) for p in points]

        x_center = ((min(x_coords) + max(x_coords)) / 2) / img_width
        y_center = ((min(y_coords) + max(y_coords)) / 2) / img_height
        width = (max(x_coords) - min(x_coords)) / img_width
        height = (max(y_coords) - min(y_coords)) / img_height

        yolo_line = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        yolo_lines.append(yolo_line)

    with open(os.path.join(output_dir, f"{img_name}.txt"), 'w') as f:
        f.write('\n'.join(yolo_lines))

print("✅ Conversion to YOLO format completed.")



In [ ]:
import shutil
import random

images_path = os.path.join(path, 'img')
dataset_path = os.path.join(path, 'dataset')

for split in ['train', 'val']:
    os.makedirs(os.path.join(dataset_path, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(dataset_path, 'labels', split), exist_ok=True)

all_images = [f for f in os.listdir(images_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(all_images)
split_index = int(len(all_images) * 0.8)
train_images = all_images[:split_index]
val_images = all_images[split_index:]

def move_files(image_list, split):
    for img_file in image_list:
        label_file = img_file.rsplit('.', 1)[0] + '.txt'
        shutil.copy2(os.path.join(images_path, img_file), os.path.join(dataset_path, 'images', split))
        if os.path.exists(os.path.join(output_dir, label_file)):
            shutil.copy2(os.path.join(output_dir, label_file), os.path.join(dataset_path, 'labels', split))

move_files(train_images, 'train')
move_files(val_images, 'val')
print("✅ Dataset organized.")



In [ ]:
data_yaml = f"""
path: {dataset_path}
train: images/train
val: images/val
names:
  0: box
"""

with open(os.path.join(path, 'dataset.yaml'), 'w') as f:
    f.write(data_yaml)

print("✅ YOLOv8 configuration file created.")

In [ ]:
from ultralytics import YOLO
import os
os.environ["WANDB_MODE"] = "disabled"

model = YOLO("yolov8n.pt")  # You can also try 'yolov8s.pt' for better accuracy
results = model.train(
    data=os.path.join(path, 'dataset.yaml'),
    epochs=50,
    batch=16,
    imgsz=640,
    workers=2,
    project='grocery-shelves',
    name='yolov8n-results',
    exist_ok=True
)

In [ ]:
metrics = model.val()
print(metrics)

In [ ]:
results = model.predict(
    source='/root/.cache/kagglehub/datasets/unidatapro/grocery-shelves/versions/1/img/',
    save=True,
    project='grocery-shelves',
    name='yolov8n-results',
    exist_ok=True
)
